In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
print("Libraries imported!")

Libraries imported!


In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Load the data
df = pd.read_csv('Training_Data_16k.csv')
print("Data loaded!")

Data loaded!


In [4]:
# Display basic information
print("Dataset Shape:", df.shape)
print("\nFirst few rows:")
print(df.head())

continuouos_features = ['Elevation_30m.tif', 'Slope_30m.tif', 'Curvature_30m.tif', 'Dis2stream_30m.tif', 'Dis2Roads_30m.tif',
                        'Rainfall_30m.tif', 'Stream_Density_30m.tif', 'SPI_30m.tif','TWI_30m.tif'
]

categorical_features = ['LULC_30m.tif', 'Geomorphology_30m.tif', 'Lithology_30m.tif', 'Soil_30m.tif'
]

circular_feature = 'Aspect_30m.tif'

# Separate features and target
X = df.drop('class', axis=1)
y = df['class']

print("\nClass distribution:")
print(y.value_counts()) #counts how many 0 and 1 classes
print("\nClass proportions:")
print(y.value_counts(normalize=True)) #gives a % of 0 and 1 classes

Dataset Shape: (16000, 15)

First few rows:
   class  Aspect_30m.tif  Curvature_30m.tif  Dis2Roads_30m.tif  \
0      0       59.036243          -0.111111         247.386337   
1      0      315.000000           0.000000        1290.000000   
2      1      116.565048          -0.222222        7122.836426   
3      1       45.000000           0.444444        1018.675598   
4      1      123.690071          -0.555556        2621.011230   

   Dis2stream_30m.tif  Elevation_30m.tif  Geomorphology_30m.tif  \
0          510.881592                 43                      1   
1          918.313660                 45                      2   
2         2002.598267                 37                      3   
3          210.000000                 39                      2   
4          954.829834                 38                      1   

   Lithology_30m.tif  LULC_30m.tif  Rainfall_30m.tif  Slope_30m.tif  \
0                  2             5       1180.459351       7.307570   
1             

In [5]:
# Replace special values with NaN
X_clean = X.replace([-128, -1, 'NA', 65535], np.nan)
X_clean = X_clean.apply(pd.to_numeric, errors='coerce')

print("\nMissing values after cleaning:")
print(X_clean.isnull().sum())

# Remove rows with missing values
mask = ~X_clean.isnull().any(axis=1)
X_clean = X_clean[mask]
y_clean = y[mask]

print(f"\nRows after removing missing values: {len(X_clean)}")
print(f"Rows removed: {len(df) - len(X_clean)}")



Missing values after cleaning:
Aspect_30m.tif            759
Curvature_30m.tif          63
Dis2Roads_30m.tif          12
Dis2stream_30m.tif         12
Elevation_30m.tif           8
Geomorphology_30m.tif       0
Lithology_30m.tif           0
LULC_30m.tif                0
Rainfall_30m.tif            9
Slope_30m.tif             166
Soil_30m.tif              196
SPI_30m.tif                78
Stream_Density_30m.tif     12
TWI_30m.tif               166
dtype: int64

Rows after removing missing values: 14930
Rows removed: 1070


In [6]:
#display of categorical features
for cat_feat in categorical_features:
    print(f"\n{cat_feat}:")
    print(X_clean[cat_feat].value_counts().sort_index())
    print(f"Number of unique categories: {X_clean[cat_feat].nunique()}")



LULC_30m.tif:
LULC_30m.tif
1      634
2      256
3        8
4       48
5    11654
6      396
7     1692
8      242
Name: count, dtype: int64
Number of unique categories: 8

Geomorphology_30m.tif:
Geomorphology_30m.tif
1    9530
2    2626
3    2323
4      40
5      11
6     400
Name: count, dtype: int64
Number of unique categories: 6

Lithology_30m.tif:
Lithology_30m.tif
1    11838
2      212
3      243
4     2610
5       27
Name: count, dtype: int64
Number of unique categories: 5

Soil_30m.tif:
Soil_30m.tif
3682.0    2677
3695.0    3935
3743.0    1989
3808.0    6329
Name: count, dtype: int64
Number of unique categories: 4


In [7]:
#continuous features 
X_continuous = X_clean[continuouos_features].copy() #copy in order not to mess up original

scaler_standard = StandardScaler()  #z-score scaling z=(x-mean)/std, mean = 0 and standard deviation = 1
X_continuous_scaled = pd.DataFrame(
    scaler_standard.fit_transform(X_continuous),  #finds mean and std of a column and scale it 
    columns=continuouos_features,
    index=X_clean.index
)

print("Continuouos features normalized (Z-score)")
print("Mean:", X_continuous_scaled.mean().round(6).values[:3], "...") #mean, rounded on 6 dec. points, shows 3 results
                                                                      #numbers should be ~0      
print("Std:", X_continuous_scaled.std().round(6).values[:3], "...")  #standard deviation -> numbers should be ~1


Continuouos features normalized (Z-score)
Mean: [-0.  0. -0.] ...
Std: [1.000033 1.000033 1.000033] ...


In [9]:
#categorical data
X_categorical = X_clean[categorical_features].copy()

for col in categorical_features:
    X_categorical[col] = X_categorical[col].astype(str)

X_categorical_encoded = pd.get_dummies(
    X_categorical, 
    columns=categorical_features,
    drop_first=True,  # Izbegava "dummy variable trap"
    prefix=categorical_features
)
#it separates types in each column into unique columns, so that column does not have values 1-6, but 0/1
#example soil has types 1,2,3. New columns: Soil_1, Soil_2, Soil_3 and each will have values 0/1 for each row

print(f"Number of columns before encoding: {len(categorical_features)}")
print(f"Number of columns after encoding: {X_categorical_encoded.shape[1]}")
print("\nNew columns(first 10):")
print(X_categorical_encoded.columns.tolist()[:10])

Number of columns before encoding: 4
Number of columns after encoding: 19

New columns(first 10):
['LULC_30m.tif_2', 'LULC_30m.tif_3', 'LULC_30m.tif_4', 'LULC_30m.tif_5', 'LULC_30m.tif_6', 'LULC_30m.tif_7', 'LULC_30m.tif_8', 'Geomorphology_30m.tif_2', 'Geomorphology_30m.tif_3', 'Geomorphology_30m.tif_4']


In [11]:
#circular data - left in degrees
aspect_values = X_clean[circular_feature].copy()

# Sin/Cos transformation (without conversion)
X_circular = pd.DataFrame({
    'Aspect_sin': np.sin(aspect_values),
    'Aspect_cos': np.cos(aspect_values)
}, index=X_clean.index)


In [ ]:
#connecting all the processed features
X_processed = pd.concat([
    X_continuous_scaled,      # Normalized continuous
    X_categorical_encoded,    # One-hot encoded categorical
    X_circular               # Sin/Cos transformed aspect
], axis=1)

print(f"\nFINAL dataset shape: {X_processed.shape}")
print(f"Sum of features: {X_processed.shape[1]}")
print(f"  - Continuous: {len(continuouos_features)}")
print(f"  - Categorical (encoded): {X_categorical_encoded.shape[1]}")
print(f"  - Circular: 2 (sin + cos)")


FINALNI dataset shape: (14930, 30)
Ukupno features: 30
  - Kontinualni: 9
  - Kategorički (encoded): 19
  - Cirkularni: 2 (sin + cos)
